In [1]:
import os
import sys
import time
import argparse
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
# from mutils import get_optimizer
from torch import optim
import torch.optim.lr_scheduler as lr_scheduler
from new_models import NLINet
from new_dataset import SNLIDataset, collate_fn, get_nli, build_vocab, load_vocab, save_vocab
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime as d
from new_train import trainepoch, evaluate
from nltk.tokenize import word_tokenize

c:\Users\desto\anaconda3\envs\ATCS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\desto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# set gpu device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
isCuda = True if device == 'cuda' else False
# Seed
seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
if isCuda:
    torch.cuda.manual_seed(seed)
val_acc_best = -1e10
stop_training = False

In [3]:
# Create vocabulary
train, valid, test = get_nli()
sentences = train['premise'] + train['hypothesis'] + \
valid['premise'] + valid['hypothesis'] + \
test['premise'] + test['hypothesis']

start_time = time.time()
word_vec = build_vocab(sentences)
print('Time to build vocab: {0}'.format(time.time() - start_time))

Found 32801 words with glove vectors
Time to build vocab: 129.52198314666748


In [11]:
# Data loaders
# train_dataset = SNLIDataset(train['premise'], train['hypothesis'], train['label'], word_vec)
# train_dataloader = DataLoader(train_dataset, batch_size= params.batch_size, collate_fn=collate_fn)
valid_dataset = SNLIDataset(valid['premise'], valid['hypothesis'], valid['label'], word_vec)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, collate_fn=collate_fn)
test_dataset = SNLIDataset(test['premise'], test['hypothesis'], test['label'], word_vec)
test_dataloader = DataLoader(test_dataset, batch_size=64, collate_fn=collate_fn)

In [6]:
# AWESentenceEncoder
savepath = 'savedir/AWESentenceEncoder_last_model.pickle'
nli_net = torch.load(savepath, map_location=torch.device(device))

print('\nTEST : Epoch {0}')
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 1e6, valid_dataloader, len(valid_dataset), 'test', True)
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 0, test_dataloader, len(test_dataset), 'test', True)


TEST : Epoch {0}


  0%|          | 0/154 [00:00<?, ?it/s]

100%|██████████| 154/154 [00:03<00:00, 51.30it/s]


finalgrep : accuracy test : 67.74


100%|██████████| 154/154 [00:03<00:00, 49.38it/s]

finalgrep : accuracy test : 67.79


(67.79, False)

In [7]:
# BLSTMEncoder max
savepath = 'savedir/BLSTMEncoder_max_model.pickle'
nli_net = torch.load(savepath, map_location=torch.device(device))

print('\nTEST : Epoch {0}')
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 1e6, valid_dataloader, len(valid_dataset), 'test', True)
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 0, test_dataloader, len(test_dataset), 'test', True)


TEST : Epoch {0}


100%|██████████| 154/154 [03:16<00:00,  1.28s/it]


finalgrep : accuracy test : 83.37


100%|██████████| 154/154 [03:04<00:00,  1.20s/it]

finalgrep : accuracy test : 83.34


(83.34, False)

In [13]:
# BLSTMEncoder last
savepath = 'savedir_saved/BLSTMEncoder_last_model.pickle'
nli_net = torch.load(savepath, map_location=torch.device(device))

print('\nTEST : Epoch {0}')
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 1e6, valid_dataloader, len(valid_dataset), 'test', True)
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 0, test_dataloader, len(test_dataset), 'test', True)


TEST : Epoch {0}


100%|██████████| 154/154 [07:30<00:00,  2.92s/it]


finalgrep : accuracy test : 76.51


100%|██████████| 154/154 [05:13<00:00,  2.03s/it]

finalgrep : accuracy test : 77.17


(77.17, False)

In [8]:
# LSTMEncoder
savepath = 'savedir/LSTMEncoder_last_model.pickle'
nli_net = torch.load(savepath, map_location=torch.device(device))

print('\nTEST : Epoch {0}')
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 1e6, valid_dataloader, len(valid_dataset), 'test', True)
evaluate(None, nli_net, None, None, val_acc_best, stop_training, 0, test_dataloader, len(test_dataset), 'test', True)


TEST : Epoch {0}


100%|██████████| 154/154 [01:26<00:00,  1.78it/s]


finalgrep : accuracy test : 76.13


100%|██████████| 154/154 [01:26<00:00,  1.78it/s]

finalgrep : accuracy test : 76.14


(76.14, False)

SentEval Multi-Task Evaluation

Loading Results

In [9]:
result_paths = {
    "AWE":"results/AWESentenceEncoder_SentEvalResults.pt",
    "LSTM":"results/LSTMEncoder_SentEvalResults.pt",
    "BiLSTM-Last":"results/BLSTMEncoder_last_SentEvalResults.pt",
    "BiLSTM-Max":"results/BLSTMEncoder_max_SentEvalResults.pt"
}

results = {key: None for key in result_paths}

for model in result_paths:
    results[model] = torch.load(result_paths[model], map_location=torch.device(device))

Micro and Macro accuracy calculations

In [10]:
def calculate_micro_macro(results):
    sum_dev = 0
    sum_samples = 0
    sum_micro = 0
    n_tasks = 0

    for task in results:
        cont = False
        if "devacc" not in results[task] or "ndev" not in results[task]:
            continue
        else:
            n_tasks+=1
        sum_dev+=results[task]["devacc"]
        sum_samples+=results[task]["ndev"]
        sum_micro+=results[task]["devacc"] * results[task]["ndev"]

    macro = 0
    if n_tasks > 0:
        macro =  (sum_dev / n_tasks)

    micro = (sum_micro / sum_samples)

    return micro, macro


micro_acc = {key: None for key in results}
macro_acc = {key: None for key in results}

for model in results:
    micro_acc[model], macro_acc[model] = calculate_micro_macro(results[model])

df = pd.DataFrame({'Model': list(results.keys()), 
                   'Micro': list(micro_acc.values()),
                   'Macro': list(macro_acc.values())})
df['Micro'] = df['Micro'].round(1)
df['Macro'] = df['Macro'].round(1)
df = df[['Model', 'Micro', 'Macro']]
display(df)

,Model,Micro,Macro
0,AWE,81.0,75.4
1,LSTM,77.6,72.8
2,BiLSTM-Last,78.9,73.0
3,BiLSTM-Max,82.3,77.1


Qualitative Analysis

In [20]:
def tokenize(sentence):
    sentence =  word_tokenize(sentence)
    token_list = []
    for token in sentence:
        token = token.lower()
        if token in word_vec:
            token_list.append(word_vec[token])
    return torch.stack(token_list, dim=0)

In [21]:
def predict_label(model_name, model, premise, hypothesis):
    # tokenize the premise and hypothesis
    premise = tokenize(premise).unsqueeze(1)
    hypothesis = tokenize( hypothesis).unsqueeze(1)

    # get the lengths
    premise_length = torch.tensor([premise.shape[1]])
    hypothesis_length = torch.tensor([hypothesis.shape[1]])

    predictions = model((premise.to(device), premise_length.to(device)), (hypothesis.to(device), hypothesis_length.to(device)))

    # get the predicted label
    predicted_label = torch.argmax(predictions, dim=1)[0]
    predicted_label = str(np.array(predicted_label.detach().cpu()))
    labels = {"0": "entailment", "1": "neutral",  "2": "contradiction"}
    predicted_label = labels[predicted_label]
    
    # return the predicted label
    return predicted_label

In [22]:
def test_all_models(premise, hypothesis):
    # predict the AWE model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modelpath = 'savedir/good_weights/AWESentenceEncoder_last_model.pickle'
    MODEL_awe = torch.load(modelpath, map_location=torch.device(device))
    awe_label = predict_label('AWE', MODEL_awe, premise, hypothesis)
    
    # predict the UniLSTM model
    modelpath = 'savedir/LSTMEncoder_last_model.pickle'
    MODEL_lstm = torch.load(modelpath, map_location=torch.device(device))
    unilstm_label = predict_label('LSTM', MODEL_lstm, premise, hypothesis)
    
    # predict the BiLSTM max model
    modelpath = 'savedir/good_weights/BLSTMEncoder_max_model.pickle'
    MODEL_bilstm_max = torch.load(modelpath, map_location=torch.device(device))
    bilstm_label = predict_label('BiLSTM_max', MODEL_bilstm_max, premise, hypothesis)
    
    # predict the BiLSTM last model
    modelpath = 'savedir/BLSTMEncoder_last_model.pickle'
    MODEL_bilstm_last = torch.load(modelpath, map_location=torch.device(device))
    bilstmmax_label = predict_label('BiLSTM_last', MODEL_bilstm_last, premise, hypothesis)
    
    # return the predicted labels
    return awe_label, unilstm_label, bilstm_label, bilstmmax_label

In [25]:
# create a list of the premises and hypothesis 
test_instances = [('Two men sitting in the sun.', 'Nobody is sitting in the shade.', 'neutral'),
                  ('A man is walking a dog” Hypothesis.', 'No cat is outside', 'neutral')]

# loop over the test instances
cell_text = []
for instance in test_instances:
    # test all models
    awe_label, unilstm_label, bilstm_label, bilstmmax_label = test_all_models(instance[0], instance[1])
    cell_text.append([awe_label, unilstm_label, bilstm_label, bilstmmax_label, instance[2]])
    
# title_text = 'Qualitative analysis of the different models'
column_headers = ['AWE', 'LSTM', 'BiLSTM_Max', 'BiLSTM_Last', 'GT']
row_headers = ['T1', 'T2']#, 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9']

plt.figure(linewidth=2, tight_layout={'pad':1})
the_table = plt.table(cellText=cell_text,
                      rowLabels=row_headers,
                      rowLoc='right',
                      colLabels=column_headers,
                      loc='center')
the_table.scale(1, 1.5)
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.box(on=None)
# plt.suptitle(title_text)
# plt.rcParams['figure.figsize'] = [20, 4]
# plt.rcParams['figure.dpi'] = 1000
plt.draw()